In [ ]:
!pip install scipy

In [ ]:
#piecewise affine

import cv2
import numpy as np
import os
import glob
from scipy.interpolate import interp1d

i = 0

# Define the function to apply the transformation to time series data
def piecewise_affine_transform(time_series, num_points=3):
    control_x = np.linspace(0, 1, num_points)
    control_y = np.random.uniform(-1, 1, num_points)
    f = interp1d(control_x, control_y, kind='linear', fill_value='extrapolate')
    transformed_time_series = time_series + f(np.linspace(0, 1, len(time_series)))
    return transformed_time_series

# Specify the input directory and output directory
input_dir = '/content/drive/MyDrive/RAW_FRISBEE_FOOTAGE'
output_dir = '/content/drive/MyDrive/AUGMENTED_FOOTAGE/NEW_PIECE'

# Iterate over the subdirectories in the input directory
for subdir, dirs, files in os.walk(input_dir):
    for file in glob.glob(subdir + '/*.mp4'):
        # Construct the output file path
        output_file = os.path.join(output_dir, os.path.basename(subdir) + '_' + os.path.basename(file))

        # Check if the output file already exists
        if os.path.exists(output_file):
            print(f"Skipping existing file: {output_file}")
            continue
        else:
            i = i+1
            print(f"Writing new file: {output_file}")
            print("wrote that shit " + str(i) + " times")
            # Open the video file
            cap = cv2.VideoCapture(file)

            # Get the dimensions of the video frames
            ret, frame = cap.read()
            rows, cols = frame.shape[0], frame.shape[1]

            # Define the codec and create a VideoWriter object
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_file, fourcc, 30.0, (cols, rows))


            # Initialize a 3D array to hold the pixel intensities for all frames
            video_data = np.zeros((30, rows, cols))

            # Read the video frames into the 3D array
            for i in range(30):
                ret, frame = cap.read()
                if ret:
                    video_data[i] = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                else:
                    break

            # Apply the transformation to each pixel's intensity values over time
            for row in range(rows):
                for col in range(cols):
                    video_data[:, row, col] = piecewise_affine_transform(video_data[:, row, col])

            # Write the transformed frames
            for i in range(30):
                frame = video_data[i].astype(np.uint8)
                frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
                out.write(frame)



            # Release everything when done
            cap.release()
            out.release()

cv2.destroyAllWindows()

In [ ]:
#superpixel

import cv2
import numpy as np
import os
import glob
from skimage.segmentation import slic
from skimage.color import label2rgb

i = 0

# Define the function to apply the superpixel transformation
def apply_superpixel_transform(image):
  # Apply the SLIC algorithm with more superpixels and lower compactness
  segments = slic(image, n_segments=1000, compactness=10, sigma=1)
  # Convert the segments to a color image
  return label2rgb(segments, image, kind='avg')


# Specify the input directory and output directory
input_dir = '/content/drive/MyDrive/RAW_FRISBEE_FOOTAGE'
output_dir = '/content/drive/MyDrive/AUGMENTED_FOOTAGE/NEW_SUPERPIXEL'

# Iterate over the subdirectories in the input directory
for subdir, dirs, files in os.walk(input_dir):
    for file in glob.glob(subdir + '/*.mp4'):
        # Construct the output file path
        output_file = os.path.join(output_dir, os.path.basename(subdir) + '_' + os.path.basename(file))

        # Check if the output file already exists
        if os.path.exists(output_file):
            print(f"Skipping existing file: {output_file}")
            continue
        else:
            # Open the video file
            cap = cv2.VideoCapture(file)

            # Get the dimensions of the video frames
            ret, frame = cap.read()
            rows, cols = frame.shape[0], frame.shape[1]

            # Get the frame rate of the input video
            fps = cap.get(cv2.CAP_PROP_FPS)

            # Define the codec and create a VideoWriter object with the same frame rate as the input video
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_file, fourcc, fps, (cols, rows))
            i = i+1
            print(f"Writing new file: {output_file}")
            print("wrote that shit " + str(i) + " times")

            while(cap.isOpened()):
                ret, frame = cap.read()
                if ret==True:
                    # Convert the frame to RGB for the superpixel transformation
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    # Apply the superpixel transformation
                    frame = apply_superpixel_transform(frame)
                    # Convert the frame back to BGR for writing
                    frame = cv2.cvtColor((frame * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)
                    # Write the transformed frame
                    out.write(frame)
                else:
                    break

            # Release everything when done
            cap.release()
            out.release()

cv2.destroyAllWindows()

In [ ]:
#gaussian blur

import cv2
import numpy as np
import os
import glob

# Define the function to apply the Gaussian blur
def apply_gaussian_blur(frame):
    return cv2.GaussianBlur(frame, (11, 11), 0)

# Specify the input directory and output directory
input_dir = '/content/onedrive/FYP/UNAUGMENTED'
output_dir = '/content/onedrive/FYP/AUGMENTED/BLUR'

i = 0

# Iterate over the subdirectories in the input directory
for subdir, dirs, files in os.walk(input_dir):
    for file in glob.glob(subdir + '/*.mp4'):
        # Construct the output file path
        output_file = os.path.join(output_dir, os.path.basename(subdir) + '_' + os.path.basename(file))

        # Check if the output file already exists
        if os.path.exists(output_file):
            print(f"Skipping existing file: {output_file}")
            continue
        else:
            # Open the video file
            cap = cv2.VideoCapture(file)

            # Get the dimensions of the video frames
            ret, frame = cap.read()
            rows, cols = frame.shape[0], frame.shape[1]

            # Get the frame rate of the input video
            fps = cap.get(cv2.CAP_PROP_FPS)

            # Define the codec and create a VideoWriter object with the same frame rate as the input video
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_file, fourcc, fps, (cols, rows))
            i = i+1
            print(f"Writing new file: {output_file}")
            print("wrote that shit " + str(i) + " times")

            while(cap.isOpened()):
                ret, frame = cap.read()
                if ret==True:
                    # Apply the Gaussian blur
                    frame = apply_gaussian_blur(frame)
                    # Write the blurred frame
                    out.write(frame)
                else:
                    break

            # Release everything when done
            cap.release()
            out.release()

cv2.destroyAllWindows()

In [ ]:
#rotation

import cv2
import numpy as np
import os
import glob
import random

# Define the function to apply the random rotation
def apply_random_rotation(frame):
    angle = random.randint(-30, 30)
    M = cv2.getRotationMatrix2D((frame.shape[1] / 2, frame.shape[0] / 2), angle, 1)
    return cv2.warpAffine(frame, M, (frame.shape[1], frame.shape[0]))

# Specify the input directory and output directory
input_dir = '/content/onedrive/FYP/UNAUGMENTED'
output_dir = '/content/onedrive/FYP/AUGMENTED/ROTATION'

i = 0

# Iterate over the subdirectories in the input directory
for subdir, dirs, files in os.walk(input_dir):
    for file in glob.glob(subdir + '/*.mp4'):
        # Construct the output file path
        output_file = os.path.join(output_dir, os.path.basename(subdir) + '_' + os.path.basename(file))

        # Check if the output file already exists
        if os.path.exists(output_file):
            print(f"Skipping existing file: {output_file}")
            continue
        else:
            # Open the video file
            cap = cv2.VideoCapture(file)

            # Get the dimensions of the video frames
            ret, frame = cap.read()
            rows, cols = frame.shape[0], frame.shape[1]

            # Get the frame rate of the input video
            fps = cap.get(cv2.CAP_PROP_FPS)

            # Define the codec and create a VideoWriter object with the same frame rate as the input video
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_file, fourcc, fps, (cols, rows))
            i = i+1
            print(f"Writing new file: {output_file}")
            print("wrote that shit " + str(i) + " times")

            while(cap.isOpened()):
                ret, frame = cap.read()
                if ret==True:
                    # Apply the random rotation
                    frame = apply_random_rotation(frame)
                    # Write the rotated frame
                    out.write(frame)
                else:
                    break

            # Release everything when done
            cap.release()
            out.release()

cv2.destroyAllWindows()

In [ ]:
#noise

import cv2
import numpy as np
import os
import glob

video_count = 0

# Define the function to apply the noise
def apply_noise(image):
    # Create a uniform noise kernel
    noise = np.random.uniform(-25, 25, image.shape).astype(np.uint8)
    # Apply the noise to the image
    noisy_image = cv2.add(image, noise)
    return noisy_image

# Specify the input directory and output directory
input_dir = '/content/onedrive/FYP/UNAUGMENTED'
output_dir = '/content/onedrive/FYP/AUGMENTED/NOISE'

# Iterate over the subdirectories in the input directory
for subdir, dirs, files in os.walk(input_dir):
    for file in glob.glob(subdir + '/*.mp4'):
        # Construct the output file path
        output_file = os.path.join(output_dir, os.path.basename(subdir) + '_' + os.path.basename(file))

        # Check if the output file already exists
        if os.path.exists(output_file):
            print(f"Skipping existing file: {output_file}")
            continue
        else:
            video_count = video_count + 1
            print(f"Writing new file: {output_file}")
            print("Processed " + str(video_count) + " videos")

            # Open the video file
            cap = cv2.VideoCapture(file)

            # Get the dimensions of the video frames
            ret, frame = cap.read()
            rows, cols = frame.shape[0], frame.shape[1]

            # Define the codec and create a VideoWriter object
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_file, fourcc, 30.0, (cols, rows))

            # Initialize a 3D array to hold the pixel intensities for all frames
            video_data = np.zeros((30, rows, cols), dtype=np.uint8)

            # Read the video frames into the 3D array
            for i in range(30):
                ret, frame = cap.read()
                if ret:
                    video_data[i] = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                else:
                    break

            # Apply the noise to each frame
            for i in range(30):
                video_data[i] = apply_noise(video_data[i])

            # Write the transformed frames
            for i in range(30):
                frame = video_data[i]
                frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
                out.write(frame)

            # Release everything when done
            cap.release()
            out.release()

cv2.destroyAllWindows()

In [ ]:
#translation

import cv2
import numpy as np
import os
import glob

video_count = 0

# Define the function to shift the image
def shift_image(image, dx, dy):
    # Define the translation matrix
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    # Use warpAffine to transform the image using the translation matrix
    shifted_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]), borderValue=(255,255,255))
    return shifted_image

# Specify the input directory and output directory
input_dir = '/content/onedrive/FYP/UNAUGMENTED'
output_dir = '/content/onedrive/FYP/AUGMENTED/TRANSLATION'

# Iterate over the subdirectories in the input directory
for subdir, dirs, files in os.walk(input_dir):
    for file in glob.glob(subdir + '/*.mp4'):
        # Construct the output file path
        output_file = os.path.join(output_dir, os.path.basename(subdir) + '_' + os.path.basename(file))

        # Check if the output file already exists
        if os.path.exists(output_file):
            print(f"Skipping existing file: {output_file}")
            continue
        else:
            video_count = video_count + 1
            print(f"Writing new file: {output_file}")
            print("Processed " + str(video_count) + " videos")

            # Open the video file
            cap = cv2.VideoCapture(file)

            # Get the dimensions of the video frames
            ret, frame = cap.read()
            rows, cols, _ = frame.shape

            # Define the codec and create a VideoWriter object
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_file, fourcc, 30.0, (cols, rows))

            # Randomly choose the direction and distance to shift for the entire video
            dx = np.random.choice([-1, 1]) * np.random.randint(0, 200)
            dy = np.random.choice([-1, 1]) * np.random.randint(0, 200)

            while True:
                ret, frame = cap.read()
                if ret:
                    # Apply the shift to each frame
                    shifted_frame = shift_image(frame, dx, dy)
                    out.write(shifted_frame)
                else:
                    break

            # Release everything when done
            cap.release()
            out.release()

cv2.destroyAllWindows()

In [ ]:
#color space transformation

import cv2
import numpy as np
import os
import glob

video_count = 0

# Define the function to shift the image
def shift_image(image, dx, dy):
    # Define the translation matrix
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    # Use warpAffine to transform the image using the translation matrix
    shifted_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]), borderValue=(255,255,255))
    return shifted_image

# Define the function to apply a color cast to the image
def color_cast(image, color_shift):
    # Split the image into its color channels
    b, g, r = cv2.split(image)
    if color_shift == 'red':
        r = r + 100
    elif color_shift == 'green':
        g = g + 100
    elif color_shift == 'blue':
        b = b + 100
    elif color_shift == 'rgb':
        r = r + 100
        g = g + 100
        b = b + 100
    # Merge the color channels back together
    return cv2.merge((b, g, r))

# Specify the input directory and output directory
input_dir = '/content/onedrive/FYP/UNAUGMENTED'
output_dir = '/content/onedrive/FYP/AUGMENTED/COLOR_TINT'

# Iterate over the subdirectories in the input directory
for subdir, dirs, files in os.walk(input_dir):
    for file in glob.glob(subdir + '/*.mp4'):
        # Construct the output file path
        output_file = os.path.join(output_dir, os.path.basename(subdir) + '_' + os.path.basename(file))

        # Check if the output file already exists
        if os.path.exists(output_file):
            print(f"Skipping existing file: {output_file}")
            continue
        else:
            video_count = video_count + 1
            print(f"Writing new file: {output_file}")
            print("Processed " + str(video_count) + " videos")

            # Open the video file
            cap = cv2.VideoCapture(file)

            # Get the dimensions of the video frames
            ret, frame = cap.read()
            rows, cols, _ = frame.shape

            # Define the codec and create a VideoWriter object
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_file, fourcc, 30.0, (cols, rows))

            # Randomly choose the direction and distance to shift for the entire video
            dx = np.random.choice([-1, 1]) * np.random.randint(0, 50)
            dy = np.random.choice([-1, 1]) * np.random.randint(0, 50)

            # Randomly choose the color shift for the entire video
            color_shift = np.random.choice(['red', 'green', 'blue', 'rgb'])

            while True:
                ret, frame = cap.read()
                if ret:
                    # Apply the shift to each frame
                    shifted_frame = shift_image(frame, dx, dy)
                    # Apply the color cast to each frame
                    color_cast_frame = color_cast(shifted_frame, color_shift)
                    out.write(color_cast_frame)
                else:
                    break

            # Release everything when done
            cap.release()
            out.release()

cv2.destroyAllWindows()

In [ ]:
#horizontal flip

import cv2
import os
import glob

video_count = 0

# Specify the input directory and output directory
input_dir = '/content/onedrive/FYP/UNAUGMENTED'
output_dir = '/content/onedrive/FYP/AUGMENTED/HORIZONTAL'

# Iterate over the subdirectories in the input directory
for subdir, dirs, files in os.walk(input_dir):
    for file in glob.glob(subdir + '/*.mp4'):
        # Construct the output file path
        output_file = os.path.join(output_dir, os.path.basename(subdir) + '_' + os.path.basename(file))

        # Check if the output file already exists
        if os.path.exists(output_file):
            print(f"Skipping existing file: {output_file}")
            continue
        else:
            video_count = video_count + 1
            print(f"Writing new file: {output_file}")
            print("Processed " + str(video_count) + " videos")

            # Open the video file
            cap = cv2.VideoCapture(file)

            # Get the dimensions of the video frames
            ret, frame = cap.read()
            rows, cols, _ = frame.shape

            # Define the codec and create a VideoWriter object
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_file, fourcc, 30.0, (cols, rows))

            while True:
                ret, frame = cap.read()
                if ret:
                    # Flip the frame horizontally
                    flipped_frame = cv2.flip(frame, 1)
                    out.write(flipped_frame)
                else:
                    break

            # Release everything when done
            cap.release()
            out.release()

cv2.destroyAllWindows()

In [ ]:
#cropping

import cv2
import numpy as np
import os
import glob

# Define the function to apply the crop
def apply_crop(frame, scale=0.75):
    height, width = frame.shape[:2]
    new_height, new_width = int(height*scale), int(width*scale)
    start_row, start_col = int((height - new_height) / 2), int((width - new_width) / 2)
    return frame[start_row:start_row+new_height, start_col:start_col+new_width]

# Specify the input directory and output directory
input_dir = '/content/onedrive/FYP/UNAUGMENTED'
output_dir = '/content/onedrive/FYP/AUGMENTED/CROPPING'

i = 0

# Iterate over the subdirectories in the input directory
for subdir, dirs, files in os.walk(input_dir):
    for file in glob.glob(subdir + '/*.mp4'):
        # Construct the output file path
        output_file = os.path.join(output_dir, os.path.basename(subdir) + '_' + os.path.basename(file))

        # Check if the output file already exists
        if os.path.exists(output_file):
            print(f"Skipping existing file: {output_file}")
            continue
        else:
            # Open the video file
            cap = cv2.VideoCapture(file)

            # Get the dimensions of the video frames
            ret, frame = cap.read()
            rows, cols = frame.shape[0], frame.shape[1]

            # Get the frame rate of the input video
            fps = cap.get(cv2.CAP_PROP_FPS)

            # Apply the crop to the first frame to get the new dimensions
            cropped_frame = apply_crop(frame)

            # Define the codec and create a VideoWriter object with the same frame rate as the input video
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_file, fourcc, fps, (cropped_frame.shape[1], cropped_frame.shape[0]))
            i = i+1
            print(f"Writing new file: {output_file}")
            print("wrote that shit " + str(i) + " times")

            while(cap.isOpened()):
                ret, frame = cap.read()
                if ret==True:
                    # Apply the crop
                    frame = apply_crop(frame)
                    # Write the cropped frame
                    out.write(frame)
                else:
                    break

            # Release everything when done
            cap.release()
            out.release()

cv2.destroyAllWindows()